In [ ]:
import os
import datajoint as dj
import pandas as pd
dj.config['database.host'] = "<host>"
dj.config['database.user'] = "<user name>"
dj.config['database.password'] = "<password>"
dj.config['enable_python_native_blobs'] = True

dj.config['schema_name'] = "<schema name>"

In [2]:
from nnfabrik.main import *

# import from your repository instead
from nnsysident.tables.experiments import TrainedModel, Experiments

Connecting konstantin@134.2.168.16:3306


# Insert entries of dataset, model and trainer that define the experiment

### Which entries from the dataset table?

In [ ]:
dataset_fn = "nnsysident.datasets.mouse_loaders.static_loaders"

dataset_paths = [
    "/notebooks/data/real/static20892-9-10-preproc0.zip",
    "/notebooks/data/real/static20892-9-11-preproc0.zip",
    "/notebooks/data/real/static20892-9-10-preproc0.zip",
    "/notebooks/data/real/static20892-9-11-preproc0.zip",
]

dataset_config_base = {
        "paths": [dataset_path],
        "seed": random_seed,
        "batch_size": 64,
        "normalize_images": True,
        "normalize_neurons": True,
        "return_more": True,
        "device": "cuda",
        "shuffle_train": True,
    }

In [ ]:
neurons_ns = [[169, 169], [164, 164], [169, 169, 169, 169], [164, 164, 164, 164]]
areas = [["V1", "PM"], ["V1", "PM"], ["V1", "PM", "AL", "RL"], ["V1", "PM", "AL", "RL"]]

dataset_fns, dataset_hashes = [], []
for dataset_path, neurons_n, area in zip(dataset_paths, neurons_ns, areas):
    
    dataset_config = dataset_config_base.copy()
    dataset_config["area"] = area
    dataset_config["neurons_n"] = neurons_n

    dataset_key = Dataset().add_entry(
        dataset_fn=dataset_fn,
        dataset_config=dataset_config,
        dataset_comment="",
        dataset_fabrikant="kklurz",
        skip_duplicates=True,
        return_pk_only=True,
    )

    dataset_fns.append(dataset_key["dataset_fn"])
    dataset_hashes.append(dataset_key["dataset_hash"])

### Which entries from the model table?

In [ ]:
model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'

model_config_base = {'init_mu_range': 0.55,
                     'init_sigma': 0.4,
                     'input_kern': 15,
                     'hidden_kern': 13,
                     'gamma_input': 1.0,
                     'grid_mean_predictor': {'type': 'cortex',
                                              'input_dimensions': 2,
                                              'hidden_layers': 0,
                                              'hidden_features': 0,
                                              'final_tanh': False},
                     'gamma_readout': 20.074,
                     'share_features': True,
                     'share_transform': False}

In [ ]:
model_fns, model_hashes = [], []

#loop over different model_configs if necessary (see dataset_config as an example)
model_config = model_config_base.copy()

model_key = Model().add_entry(
    model_fn=model_fn,
    model_config=model_config,
    model_comment="",
    model_fabrikant="kklurz",
    skip_duplicates=True,
    return_pk_only=True,
)

model_fns.append(model_key["model_fn"])
model_hashes.append(model_key["model_hash"])

### Which entries from the trainer table?

In [ ]:
trainer_fn = "nnsysident.training.trainers.standard_trainer"

trainer_config_base = {'detach_core': True}

In [ ]:
trainer_fns, trainer_hashes = [], []

#loop over different trainer_configs if necessary (see dataset_config as an example)
trainer_config = trainer_config_base.copy()

trainer_key = Trainer().add_entry(
    trainer_fn=trainer_fn,
    trainer_config=trainer_config,
    trainer_comment="",
    trainer_fabrikant="kklurz",
    skip_duplicates=True,
    return_pk_only=True,
)

trainer_fns.append(trainer_key["trainer_fn"])
trainer_hashes.append(trainer_key["trainer_hash"])

# Get the restrictions of dataset, model and trainer which make up the experiment

In [ ]:
dfs, dhs, mfs, mhs, tfs, ths = [], [], [], [], [], []
restrictions = []
for (df, dh), (mf, mh), (tf, th) in product(
    zip(dataset_fns, dataset_hashes),
    zip(model_fns, model_hashes),
    zip(trainer_fns, trainer_hashes),
):
    combination = {
        "dataset_fn": df,
        "dataset_hash": dh,
        "model_fn": mf,
        "model_hash": mh,
        "trainer_fn": tf,
        "trainer_hash": th,
    }
    restrictions.append(combination)

# Add the experiment to the table

In [ ]:
fabrikant_name = "kklurz"
experiment_name = "My insightful experiment"
experiment_comment = "This experiment analyzes ..."

Experiments().add_entry(
    experiment_name=experiment_name,
    experiment_fabrikant=fabrikant_name,
    experiment_comment=experiment_comment,
    restrictions=restrictions,
    skip_duplicates=True,
)

# Check out the progress of your experiment

In [4]:
experiment_name = "Linear model, gaussian and factorized readout, 20457-5-9"
tm_restriction = Experiments().Restrictions() & 'experiment_name="{}"'.format(
    experiment_name
)

TrainedModel.progress(tm_restriction)

TrainedModel         Completed 10 of 10 (100.0%)   2021-04-13 14:12:20


(0, 10)

# Populate your experiment(s)

In [7]:
for experiment_name in ["My insightful experiment", "My other insightful experiment"]:

    TrainedModel.populate(Experiments.Restrictions & 'experiment_name="{}"'.format(experiment_name),
                          reserve_jobs=True,
                          order="random",)

# Fetch your finished experiment

In [9]:
import pandas as pd

In [14]:
experiment_name = "Linear model, gaussian and factorized readout, 20457-5-9"

data = pd.DataFrame(((TrainedModel * 
                       Experiments.Restrictions & 
                       'experiment_name="{}"'.format(experiment_name)) *
                      Dataset *
                      Model *
                      Trainer *
                      Seed).fetch())

# Plot stuff...

---

## Monitor experiments and table populations

In [16]:
tms_n = 0
for experiment_name in Experiments.fetch("experiment_name"):
    print(experiment_name)
    tm_restriction = Experiments.Restrictions & 'experiment_name="{}"'.format(
        experiment_name
    )
    aa = TrainedModel.progress(tm_restriction)
    tms_n += aa[1]
    print("\n")

Linear model, gaussian and factorized readout, 20457-5-9
TrainedModel         Completed 10 of 10 (100.0%)   2021-04-13 14:20:03


Real, Direct, grid_mean_predictor=False, se2d_fullgaussian2d, 4-set
TrainedModel         Completed 150 of 150 (100.0%)   2021-04-13 14:20:07


Real, Direct, grid_mean_predictor=False, share_features=False, se2d_fullgaussian2d, 4-set
TrainedModel         Completed 150 of 150 (100.0%)   2021-04-13 14:20:12


Real, Direct, random 128, se2d_fullgaussian2d, 20457-5-9
TrainedModel         Completed 150 of 150 (100.0%)   2021-04-13 14:20:17


Real, Direct, random 64, se2d_fullgaussian2d, 20457-5-9
TrainedModel         Completed 150 of 150 (100.0%)   2021-04-13 14:20:21


Real, Direct, se2d_fullgaussian2d, 1-S
TrainedModel         Completed 5 of 5 (100.0%)   2021-04-13 14:20:25


Real, Direct, se2d_fullgaussian2d, 11-S
TrainedModel         Completed 5 of 5 (100.0%)   2021-04-13 14:20:30


Real, Direct, se2d_fullgaussian2d, 20457-5-9
TrainedModel         Completed 30

KeyboardInterrupt: 